In [47]:
# criar um navegador
import re
import pandas as pd
from time import sleep
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# importar/visualizar a base de dados

df_buscas = pd.read_excel('buscas.xlsx')   
display(df_buscas)

,Nome,Termos banidos,Preço mínimo,Preço máximo
0,Iphone 16 128 gb,mini watch vitrine usado,2000,7500
1,rtx 3060,zota galax,1700,3500


In [59]:
def find_google(driver, product, banned_terms, min_price, max_price):
    list_google = []

    banned_terms = banned_terms.lower()
    list_banned_terms = banned_terms.split(' ')
    list_product_terms = product.split(' ')

    # Pesquisar o nome do produto no Google
    driver.get('https://www.google.com.br')
    sleep(1)
    # Localizar a caixa de pesquisa e enviar o nome do produto
    driver.find_element(By.CLASS_NAME, 'gLFyf').send_keys(product, Keys.ENTER)

    sleep(1)
    list_items = driver.find_elements(By.XPATH, '//div[@role="listitem"]')

    sleep(1)
    for item in list_items:
        if 'Shopping' in item.text:
            item.click()
            break
    sleep(1)
    # Pegar a lista de resultados da busca no Google Shopping
    list_results = WebDriverWait(driver, 2).until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'i0X6df')))

    # Processa os resultados
    for result in list_results:
        try:
            # Pega o nome do produto
            name = WebDriverWait(result, 2).until(EC.presence_of_element_located((By.TAG_NAME, 'h3'))).text.lower()

            # Verifica se o nome contém termos banidos
            banned_terms_found = False
            for banned_term in list_banned_terms:
                if banned_term in name:
                    banned_terms_found = True
                    break

            # Verifica se o nome contém todos os termos do produto
            product_terms_found = True
            for product_term in list_product_terms:
                if product_term not in name:
                    product_terms_found = False
                    break

            # Se o nome não contém termos banidos e contém os termos do produto
            if not banned_terms_found and product_terms_found:
                # Pega o preço do produto
                price_element = WebDriverWait(result, 2).until(EC.presence_of_element_located((By.CLASS_NAME, 'a8Pemb')))
                price_text = price_element.text
                price = float(price_text.replace('R$', '').replace(' ', '').replace('.', '').replace(',', '.'))
                    #price = float(re.sub('[^0-9,.]', '', price_text).replace('R$', '').replace(' ', '').replace('.', '').replace(',', '.'))
                

                # Verifica se o preço está dentro da faixa desejada
                if min_price <= price <= max_price:
                    # Pega o link do produto
                    element_ref = result.find_element(By.CLASS_NAME, 'bONr3b')
                    elemento_pai = element_ref.find_element(By.XPATH, '..')  
                    link = elemento_pai.get_attribute('href')
                    
                    list_google.append((name, price, link))

        except Exception as e:
            print(f"Erro ao processar item: {e}")
            continue

    return list_google


def find_buscape(driver, product, banned_terms, min_price, max_price):
    list_buscape = []

    banned_terms = banned_terms.lower()
    list_banned_terms = banned_terms.split(' ')
    list_product_terms = product.split(' ')

    # Pesquisar o nome do produto no Google
    driver.get('https://www.buscape.com.br/')
    sleep(1)
    # Localizar a caixa de pesquisa e enviar o nome do produto
    driver.find_element(By.XPATH, '//*[@id="new-header"]/div[1]/div/div/div[3]/div/div/div[2]/div/div[1]/input').send_keys(product, Keys.ENTER)

    sleep(1)
    # Pegar a lista de resultados da busca no Google Shopping
    list_results = WebDriverWait(driver, 2).until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'Hits_ProductCard__Bonl_')))

    # Processa os resultados
    for result in list_results:
        try:
            # Pega o nome do produto
            name = WebDriverWait(result, 2).until(EC.presence_of_element_located((By.XPATH, '//*[@data-testid="product-card::name"]'))).text.lower()

            # Verifica se o nome contém termos banidos
            banned_terms_found = False
            for banned_term in list_banned_terms:
                if banned_term in name:
                    banned_terms_found = True
                    break

            # Verifica se o nome contém todos os termos do produto
            product_terms_found = True
            for product_term in list_product_terms:
                if product_term not in name:
                    product_terms_found = False
                    break

            # Se o nome não contém termos banidos e contém os termos do produto
            if not banned_terms_found and product_terms_found:
                # Pega o preço do produto
                price_element = WebDriverWait(result, 2).until(EC.presence_of_element_located((By.XPATH, '//*[@data-testid="product-card::price"]')))
                price_text = price_element.text
                price = float(price_text.replace('R$', '').replace(' ', '').replace('.', '').replace(',', '.'))
                #price = float(re.sub('[^0-9,.]', '', price_text).replace('R$', '').replace(' ', '').replace('.', '').replace(',', '.'))
                            
            
                # Verifica se o preço está dentro da faixa desejada
                
                if min_price <= price <= max_price:
                    # Pega o link do produto
                    link  = result.find_element(By.XPATH, '//a[@data-testid="product-card::card"]').get_attribute('href')
                    #elemento_pai = element_ref.find_element(By.XPATH, '..')  
                    #link = elemento_pai.get_attribute('href')
                    
                    list_buscape.append((name, price, link))    

        except Exception as e:
            print(f"Erro ao processar item: {e}")
            continue

    return list_buscape


In [49]:
# Criar o DataFrame para armazenar os resultados
df_complete = pd.DataFrame()

# Criar o driver do Selenium
driver = webdriver.Chrome()

# Loop para processar cada item do df_buscas
for item in df_buscas.index:
    # Obtendo os dados para a busca
    product = df_buscas.loc[item, 'Nome']
    banned_terms = df_buscas.loc[item, 'Termos banidos']
    min_price = df_buscas.loc[item, 'Preço mínimo']
    max_price = df_buscas.loc[item, 'Preço máximo']

    print(f"\nProcessando item {item + 1}: {product}")  # Debug: Verificar qual item está sendo processado

    # Buscar resultados no Google
    list_google = find_google(driver, product, banned_terms, min_price, max_price)
    print(f"Resultados do Google: {list_google}")  # Debug: Verificar os resultados do Google
    if list_google:
        # Criar DataFrame a partir dos resultados do Google
        df_google = pd.DataFrame(list_google, columns=['Nome', 'Preço', 'Link'])
        df_complete = pd.concat([df_complete, df_google], ignore_index=True)
    else:
        print("Nenhum resultado do Google encontrado.")  # Debug
    
    
    # Buscar resultados no Buscapé
    list_buscape = find_buscape(driver, product, banned_terms, min_price, max_price)
    print(f"Resultados do Buscapé: {list_buscape}")  # Debug: Verificar os resultados do Buscapé
    if list_buscape:
        # Criar DataFrame a partir dos resultados do Buscapé
        df_buscape = pd.DataFrame(list_buscape, columns=['Nome', 'Preço', 'Link'])
        # Concatenar ao DataFrame acumulador
        df_complete = pd.concat([df_complete, df_buscape], ignore_index=True)
    else:
        print("Nenhum resultado do Buscapé encontrado.")  # Debug

# Fechar o driver do Selenium
driver.quit()

# Exibir o DataFrame final
print("Resultados finais combinados:")



Processando item 1: Iphone 16 128 gb
Resultados do Google: []
Nenhum resultado do Google encontrado.
Resultados do Buscapé: []
Nenhum resultado do Buscapé encontrado.

Processando item 2: rtx 3060
Resultados do Google: [('placa de vídeo rtx 3060 12gb twin x2 gddr6 inno3d - n30602-12d6-119032ah', 1968.99, 'https://www.google.com.br/url?url=https://www.pichau.com.br/placa-de-video-inno3d-geforce-rtx-3060-twin-x2-12gb-gddr6-192-bit-n30602-12d6-119032ah%3Fsrsltid%3DAfmBOooh9JYqqGMLEvLphLRJpZO8Qhxekdf80pZsMU2IoEwSjr0oMwb192I&rct=j&q=&esrc=s&opi=95576897&sa=U&ved=0ahUKEwi_rJelsvCKAxUGkZUCHRRrCbsQguUECOQL&usg=AOvVaw3t-priiuhV7lX6GFayCw4Q'), ('placa de vídeo gainward geforce rtx 3060 12gb ghost gddr6 - ne63060019k9-190au', 1971.0, 'https://www.google.com.br/url?url=https://www.kabum.com.br/produto/185626/placa-de-video-rtx-3060-gainward-nvidia-geforce-ghost-190au-12gb-gddr6-192-bits-dual-fan-ne63060019k9%3Fsrsltid%3DAfmBOopeeVvn8vBEsmu5G-qMqC7UNYUrhhMawdHGBgxIESOMmiV_5gKRhY0&rct=j&q=&esrc=s&o

In [50]:
display(df_complete)

,Nome,Preço,Link
0,placa de vídeo rtx 3060 12gb twin x2 gddr6 inn...,1968.99,https://www.google.com.br/url?url=https://www....
1,placa de vídeo gainward geforce rtx 3060 12gb ...,1971.00,https://www.google.com.br/url?url=https://www....
2,placa de vídeo palit geforce rtx 3060 dual 12g...,1799.90,https://www.google.com.br/url?url=https://www....
3,placa de vídeo msi geforce rtx 3060 ventus 2x ...,2339.00,https://www.google.com.br/url?url=https://www....
4,placa de video igame geforce rtx 3060 mini 192...,2599.00,https://www.google.com.br/url?url=https://www....
5,placa de vídeo manli nvidia geforce rtx 3060 1...,2099.50,https://www.google.com.br/url?url=https://www....
6,placa de vídeo colorful geforce rtx 3060 nb du...,1899.99,https://www.google.com.br/url?url=https://www....
7,placa de video asus geforce rtx 3060 12gb gddr...,3391.08,https://www.google.com.br/url?url=https://www....
8,"placa de vídeo rtx 3060 pcyes nvidia geforce, ...",2099.00,https://www.google.com.br/url?url=https://shop...
9,nvidia | acessórios | placa de vídeo nvidia ge...,1825.00,https://www.google.com.br/url?url=https://www....


In [53]:
display(df_buscas)

,Nome,Termos banidos,Preço mínimo,Preço máximo
0,Iphone 16 128 gb,mini watch vitrine usado,2000,7500
1,rtx 3060,zota galax,1700,3500


In [60]:
# criar o navegador
driver = webdriver.Chrome()
# tratar os valores que vieram da tabela
product = 'Iphone 14 128 gb'
banned_terms = 'mmini watch'
min_price = 1000
max_price = 10000

# chamar a função 
list_google = find_google(driver, product, banned_terms, min_price, max_price)

list_buscape = find_buscape(driver, product, banned_terms, min_price, max_price)

driver.quit()


# Criar DataFrames de exemplo
df_google = pd.DataFrame(list_google, columns=['Nome', 'Preço', 'Link'])
df_buscape = pd.DataFrame(list_buscape, columns=['Nome', 'Preço', 'Link'])

# Combinar os DataFrames usando a coluna 'Nome' como chave
#df_complet = pd.merge(df_buscape, df_google, on='Nome', how='inner', suffixes=('_buscape', '_google'))

# Exibir o DataFrame combinado
display(df_google, df_buscape)

,Nome,Preço,Link


,Nome,Preço,Link
